In [1]:
print('hello world')

hello world


In [2]:
import re
import docx2txt
from pdfminer.high_level import extract_text
from sklearn.metrics.pairwise import cosine_similarity
from gensim.models import Word2Vec
from nltk.tokenize import word_tokenize
import numpy as np
import nltk

nltk.download('punkt')


[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Affan\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [3]:
resume = input("Enter the paths to your resume files, separated by commas: ").split(',')
filetype= input("Enter the file type of your resumes (word/pdf): ").strip().lower()

#  job description
jobdescription = input("Enter the job description text: ")

Extracting Texts from input data

In [4]:
# extracting text from word doc
def extract_text_from_word(file_path):
    return docx2txt.process(file_path)

# extracting from pdf
def extract_text_from_pdf(file_path):
    return extract_text(file_path)


Extracting Skills, Education, Experience

In [5]:
# extracting skills education years of experience
def extract_resume_sections(text):
    skills = extract_skills(text)
    education = extract_education(text)
    experience = extract_experience(text)
    return skills, education, experience
# skills
# Function to extract skills
def extract_skills(text):
    skills_pattern = re.compile(r"(skills|technical skills|programming languages)(.*?)(education|experience|work experience)", re.IGNORECASE | re.DOTALL)
    skills = skills_pattern.search(text)
    return skills.group(2).strip() if skills else ""

# education
# Function to extract education
def extract_education(text):
    education_pattern = re.compile(r"(education)(.*?)(experience|skills|work experience)", re.IGNORECASE | re.DOTALL)
    education = education_pattern.search(text)
    return education.group(2).strip() if education else ""

# experience
# Function to extract years of experience
def extract_experience(text):
    experience_pattern = re.compile(r"(experience|work experience)(.*)", re.IGNORECASE | re.DOTALL)
    experience = experience_pattern.search(text)
    return experience.group(2).strip() if experience else ""


Tokenization

In [6]:
# making tokens of input texts
def tokenize(text):
    return word_tokenize(text.lower())


Fitting Word2Vec

In [7]:
# Function to calculate average Word2Vec vector for a section of text
def average_word2vec_vector(model, text):
    tokens = tokenize(text)
    vectors = [model.wv[word] for word in tokens if word in model.wv]
    if len(vectors) == 0:
        return np.zeros(model.vector_size)    #making vectors of skills education and experince f founf if not found return zero(0)
    return np.mean(vectors, axis=0)


Cosine Similarity

In [8]:
# measuring cosine similarity accross different vectors
def calculate_similarity_with_word2vec(model, resume_section, job_section):
    resume_vector = average_word2vec_vector(model, resume_section)
    job_vector = average_word2vec_vector(model, job_section)
    return cosine_similarity([resume_vector], [job_vector])[0][0]


Inputs

In [9]:
#pdfs
resume_files = resume
file_type = filetype

#  job description
job_description = jobdescription

In [10]:
print(resume_files)
print(file_type)
print(job_description)

['2.pdf', 'affangohar_cv.pdf']
pdf
LN Technologies, a subsidiary of App Maisters Inc USA, is an Enterprise App Development company that specializes in developing Mobility, Web, BI and Cloud based solutions. With over 400 successful projects under our portfolio, we take pride in catering to the needs of all sizes and types of clients, from ambitious startups & small businesses to large multinationals.  We are currently seeking highly motivated Engineers in Karachi office to join our team. The ideal candidates should have a strong technical background, possess the ability to work independently as well as collaboratively, and have some experience in designing, developing, testing, and implementing engineering projects. The engineers will play a key role in building AI applications and will be given significant responsibility within a small startup environment.  Job Requirements:  Moderate experience in Python: Proficiency in Python programming language. Demonstrated experience in developi

Concat texts of Job description and Input CV's

In [11]:
# Combine all text from the resumes and job description to train Word2Vec model
all_text = job_description

resume_texts = []
for resume_file in resume_files:
    if file_type == 'word':
        resume_text = extract_text_from_word(resume_file.strip())
    elif file_type == 'pdf':
        resume_text = extract_text_from_pdf(resume_file.strip())
    else:
        print("Unsupported file type!")
        exit()

    resume_texts.append(resume_text)
    all_text += " " + resume_text
# Tokenize and train Word2Vec model
tokenized_text = [tokenize(all_text)]
model = Word2Vec(sentences=tokenized_text, vector_size=100, window=5, min_count=1, workers=4)

# Extract job description sections
job_skills, job_education, job_experience = extract_resume_sections(job_description)

# Initialize a dictionary to store overall similarity scores for each resume
resume_scores = {}

# For each resume, calculate similarity for skills, education, and experience
for i, resume_text in enumerate(resume_texts):
    resume_skills, resume_education, resume_experience = extract_resume_sections(resume_text)

    # Calculate similarity scores for skills, education, and experience
    skills_similarity = calculate_similarity_with_word2vec(model, resume_skills, job_skills)
    education_similarity = calculate_similarity_with_word2vec(model, resume_education, job_education)
    experience_similarity = calculate_similarity_with_word2vec(model, resume_experience, job_experience)

    # Combine the scores (you can weigh them differently if necessary)
    overall_score = (skills_similarity + education_similarity + experience_similarity) / 3
    resume_scores[resume_files[i].strip()] = overall_score

    print(f"Resume {i+1} - {resume_files[i].strip()} similarity scores:")
    print(f"  Skills Similarity: {skills_similarity:.2f}")
    print(f"  Education Similarity: {education_similarity:.2f}")
    print(f"  Experience Similarity: {experience_similarity:.2f}")
    print(f"  Overall Similarity: {overall_score:.2f}\n")

# Suggest the best resume
best_resume = max(resume_scores, key=resume_scores.get)
print(f"The best matching resume for the job description is: {best_resume}")

LookupError: 
**********************************************************************
  Resource [93mpunkt_tab[0m not found.
  Please use the NLTK Downloader to obtain the resource:

  [31m>>> import nltk
  >>> nltk.download('punkt_tab')
  [0m
  For more information see: https://www.nltk.org/data.html

  Attempted to load [93mtokenizers/punkt_tab/english/[0m

  Searched in:
    - 'C:\\Users\\Affan/nltk_data'
    - 'c:\\Users\\Affan\\AppData\\Local\\Programs\\Python\\Python312\\nltk_data'
    - 'c:\\Users\\Affan\\AppData\\Local\\Programs\\Python\\Python312\\share\\nltk_data'
    - 'c:\\Users\\Affan\\AppData\\Local\\Programs\\Python\\Python312\\lib\\nltk_data'
    - 'C:\\Users\\Affan\\AppData\\Roaming\\nltk_data'
    - 'C:\\nltk_data'
    - 'D:\\nltk_data'
    - 'E:\\nltk_data'
**********************************************************************


Model Training and Output 

In [ ]:
# # Tokenize and train Word2Vec model
# tokenized_text = [tokenize(all_text)]
# model = Word2Vec(sentences=tokenized_text, vector_size=100, window=5, min_count=1, workers=4)

# # Extract job description sections
# job_skills, job_education, job_experience = extract_resume_sections(job_description)

# # Initialize a dictionary to store overall similarity scores for each resume
# resume_scores = {}

# # For each resume, calculate similarity for skills, education, and experience
# for i, resume_text in enumerate(resume_texts):
#     resume_skills, resume_education, resume_experience = extract_resume_sections(resume_text)

#     # Calculate similarity scores for skills, education, and experience
#     skills_similarity = calculate_similarity_with_word2vec(model, resume_skills, job_skills)
#     education_similarity = calculate_similarity_with_word2vec(model, resume_education, job_education)
#     experience_similarity = calculate_similarity_with_word2vec(model, resume_experience, job_experience)

#     # Combine the scores (you can weigh them differently if necessary)
#     overall_score = (skills_similarity + education_similarity + experience_similarity) / 3
#     resume_scores[resume_files[i].strip()] = overall_score

#     print(f"Resume {i+1} - {resume_files[i].strip()} similarity scores:")
#     print(f"  Skills Similarity: {skills_similarity:.2f}")
#     print(f"  Education Similarity: {education_similarity:.2f}")
#     print(f"  Experience Similarity: {experience_similarity:.2f}")
#     print(f"  Overall Similarity: {overall_score:.2f}\n")

# # Suggest the best resume
# best_resume = max(resume_scores, key=resume_scores.get)
# print(f"The best matching resume for the job description is: {best_resume}")

Best Suggested Resume/CV

In [ ]:
# # best CV/Resume
# best_resume = max(resume_scores, key=resume_scores.get)
# print(f"The best matching resume for the job description is: {best_resume}")